In [53]:
import pandas as pd
import sys
reload(sys)
sys.setdefaultencoding('utf8')


In [54]:
test = True

if(test):
    NOME_FILE = "test_dataset.klp"
    path = "../test_ironita2018_revnlt_processed.tsv"
else:
    NOME_FILE = "train_dataset.klp"
    path = "../training_ironita2018_renlt_processed.tsv"
    #path = "../training_ironita2018.tsv"

df = pd.read_csv(path, sep='\t')
df.head(3)


,id,text,topic,text::text::S
0,595524450503815168,-Prendere i libri in copisteria-Fare la spesa-Spararmi in bocca-Farmi la doccia,TWITA,-Prendere::-prendere::V i::i::RD libri::libro::S in::in::E copisteria-Fare::copisteria-fare::V la::la::RD spesa-Spararmi::spesa-spararmi::S in::in::E bocca-Farmi::bocca-farmi::S la::la::RD doccia::doccia::S
1,578468106504433665,...comunque con una crociera Costa se non ti ammazza Schettino prima ti spara il terrorista dopo...,HSC,.::.::FF comunque::comunque::B con::con::E una::una::RI crociera::crociera::S Costa::costa::SP se::se::PC non::non::BN ti::ti::PC ammazza::ammazza::V Schettino::schettino::SP prima::prima::B ti::ti::PC spara::sparare::V il::il::RD terrorista::terrorista::S dopo::dopo::B .::.::FS
2,577791521174466560,"“@TweetComici: Ogni ragazza: \""non sono una ragazza gelosa.\""*3 minuti dopo*\""CHI CAZZO È QUELLA PUTTANA?\""”",TWITA,"“::“::FB @TweetComici::__user::USR __COLON__::__COLON__::FC Ogni::ogni::SP ragazza::ragazzo::S __COLON__::__COLON__::FC \::\::FB ""::""::FB non::non::BN sono::essere::V una::una::RI ragazza::ragazzo::S gelosa::geloso::A .::.::FF \::\::FB ""::""::FB *::*::FB 3::3::N minuti::minuto::S dopo*\::dopo*\::A ""::""::FB CHI::chi::PR cazzo::cazzo::A È::essere::V quella::quello::PD puttana::puttana::A ?::?::FS..."


# Exploratory Analysis

In [55]:
df.info()

In [82]:
if(not test):
    pd.options.display.max_colwidth = 100
    df[(df.irony==0) & (df.sarcasm ==1)].head(3)

In [83]:
if(not test):   
    pd.options.display.max_colwidth = 400
    df[(df.irony==1) | (df.sarcasm ==1)][['text','irony','sarcasm']].sample(20)

In [84]:
if(not test):
    df[(df.irony==1) | (df.sarcasm ==1)].count()

### Generazione file KLP

In [59]:
def generaArraydataset(df=df, target="irony", lower = True):
    arrayDataset = []
    arrayTarget = []
    for index, row in df.iterrows():
        if(lower):
            arrayDataset.append(row["text"].lower())
        else:
            arrayDataset.append(row["text"])
        try:
            if(row[target] == 0):
                arrayTarget.append(-1)
            else:
                arrayTarget.append(row[target])
        except:
            arrayTarget.append(-1)
    return [arrayDataset, arrayTarget]

array = generaArraydataset()
X_train = array[0]
y_train_irony = array[1]
y_train_sarcasm = generaArraydataset(target="sarcasm")[1]

'''
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True)

#token = nltk.word_tokenize(X_train[2])
print(tknzr.tokenize(X_train[2]))
'''
print(y_train_irony[2])
print(y_train_sarcasm[2])

In [61]:
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
import hashlib

def creaBoW(dataset, target, name = "dataset.klp"):
    f = open(name,"w+")
    for index, riga in enumerate(dataset):
        string = str(target[index]) + " |BV:bow| "
        tknzr = TweetTokenizer(strip_handles=True)
        tweettoknz = tknzr.tokenize(dataset[index])
        for word in tweettoknz:
            string = string + "_" + hashlib.md5(''.join(word)).hexdigest() + ":1.0 "
        f.write(string + "|EV| |BS:comment| "+ dataset[index] +" |ES| |BS:IDTweet| "+ str(df.iloc[index]['id']) +" |ES| \n")    
    f.close()

    
def creaBoW2(df=df, name = "dataset.klp", test=False):
    f = open(name,"w+")
    for index, row in df.iterrows():
        if(test):
            try:
                if(row["irony"] == 1):
                    ironia = "Irony"
            except:
                ironia = "NOTIrony"
            try:    
                if(row["sarcasm"] == 1):
                    sarcasmo = "Sarcasmo"
            except:
                sarcasmo = "NOTSarcasmo"
        else:
            if(row["irony"] == 1):
                ironia = "Irony"
            else:
                ironia = "NOTIrony"
            if(row["sarcasm"] == 1):
                sarcasmo = "Sarcasmo"
            else:
                sarcasmo = "NOTSarcasmo"
        string = ironia + " " + sarcasmo + " |BV:bow| "
        tknzr = TweetTokenizer(strip_handles=True)
        tweettoknz = tknzr.tokenize(row["text"])
        for word in tweettoknz:
            string = string + "_" + hashlib.md5(''.join(word)).hexdigest() + ":1.0 "
        #f.write(string + "|EV| |BS:comment| "+ row["text"] +" |ES| |BS:IDTweet| "+ str(row["id"]) +" |ES| \n")
        f.write(string + "|EV| |BS:IDTweet| "+ str(row["id"]) +" |ES| \n") 
    f.close()
    

#creaBoW(X_train, y_train_irony, NOME_FILE)
creaBoW2(name = NOME_FILE, test=test)
#NOME_FILE = "dataset_sarcasmo.klp"
#creaBoW(X_train, y_train_sarcasm, NOME_FILE)


In [62]:
#df = pd.read_csv("../training_ironita2018_renlt_processed.tsv", sep='\t')
#df.head(3)


In [63]:
def generaArrayTokenizzatoLemmi(df=df):
    arrayLemmi = []
    for index, row in df.iterrows():
        arrayLemmi.append(row["text::text::S "])
    arrayTokenizzatoLemmi = []
    #per ogni elemento del dataset
    for text in arrayLemmi:
        arrayParoleLemm = text.split()
        arrayFrase = []
        #per ogni token text::text::S
        for parola in arrayParoleLemm:
            arrayParola = parola.split("::")
            #divido ogni singolo token e prendo il lemma
            arrayFrase.append(arrayParola[1])
        arrayTokenizzatoLemmi.append(arrayFrase)
    return arrayTokenizzatoLemmi

def generaArrayTokenizzatoSurface(df=df, listPOS = ['A','B','S','V','HTG'], lower = True):
    arrayLemmi = []
    for index, row in df.iterrows():
        arrayLemmi.append(row["text::text::S "])
    arrayTokenizzatoLemmi = []
    for text in arrayLemmi:
        arrayParoleLemm = text.split()
        arrayFrase = []
        for parola in arrayParoleLemm:
            arrayParola = parola.split("::")
            if((arrayParola[2] in listPOS) and listPOS != []):
                if(lower):
                    arrayFrase.append(arrayParola[0].lower())
                else:
                    arrayFrase.append(arrayParola[0])
            if(listPOS == []):
                if(lower):
                    arrayFrase.append(arrayParola[0].lower())
                else:
                    arrayFrase.append(arrayParola[0])
        arrayTokenizzatoLemmi.append(arrayFrase)
    return arrayTokenizzatoLemmi
    
prova = generaArrayTokenizzatoLemmi()
print prova[2]
prova2 = generaArrayTokenizzatoSurface()
print prova2[2]
arrayCompleto = generaArrayTokenizzatoSurface(listPOS = [])
print arrayCompleto[2]

In [64]:
import nltk
from nltk.util import ngrams

def generaArrayNgrams(dataset, ngram=2):
    arrayBigrammi = []
    for item in dataset:
        arrayBigrammi.append(list(ngrams(item,ngram)))
    return arrayBigrammi


arrayBigrammiLemmi = generaArrayNgrams(prova)
arrayBigrammiSurface = generaArrayNgrams(arrayCompleto)


arrayBigrammiChar = generaArrayNgrams(X_train)
array3grammiChar = generaArrayNgrams(X_train, 3)
array4grammiChar = generaArrayNgrams(X_train, 4)
array5grammiChar = generaArrayNgrams(X_train, 5)

In [65]:
import numpy as np
from numpy import asarray

def importaWordSpace(pathWS = "../ws"):
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open(pathWS)
    for line in f:
        values = line.split()
        word = values[0]
        #print(word)
        string = ''.join(values[3:]).split(',')
        for x in string:
            x = float(x)
        #print(string)
        coefs = asarray(string, dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index

def importaWordSpace2(pathWS = '../DPL-IT_lrec2016.txt'):
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open(pathWS)
    for line in f:
        values = line.split()
        word = values[0]
        #print(word)
        string = ''.join(values[1:]).split(',')
        for x in string:
            x = float(x)
        #print(string)
        coefs = asarray(string, dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index

def importaWordSpace3(pathWS = '../emoji2vec.txt'):
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open(pathWS)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index

def importaWordSpace4(pathWS = '../sentix'):
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open(pathWS)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[3:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index

def generaCombWordSpace(dataset = prova2):
    embeddings_index = importaWordSpace()
    combDataset = []
    for tweet in dataset:
        comb = np.zeros(250)
        norm = 0.0
        for parola in tweet:
            if(parola in embeddings_index):
                comb = comb + embeddings_index[parola]
                norm += 1
        if(norm != 0):
            combDataset.append(comb/norm)
        else:
            combDataset.append(comb)
    return combDataset

def generaCombDPL2(df=df):
    arrayLemmi = []
    embeddings_index = importaWordSpace2()
    for index, row in df.iterrows():
        arrayLemmi.append(row["text::text::S "])
    arrayTokenizzatoLemmi = []
    for text in arrayLemmi:
        arraySmooth = [0.3333 , 0.3333 , 0.3333]
        arNomi = np.zeros(3)
        cnomi = 0
        arVerb = np.zeros(3)
        cverb = 0
        arAdj = np.zeros(3)
        cadj = 0
        arAdv = np.zeros(3)
        cadv = 0
        arTOT = np.zeros(3)
        ctot = 0
        arrayParoleLemm = text.split()
        arrayFrase = []
        for parola in arrayParoleLemm:
            arrayParola = parola.split("::")
            if(arrayParola[2] == 'A'):
                try:
                    arAdj += embeddings_index[arrayParola[0].lower()]
                    arTOT += embeddings_index[arrayParola[0].lower()]
                    cadj += 1
                    ctot += 1
                except:
                    arAdj += arraySmooth
                    arTOT += arraySmooth
                    cadj += 1
                    ctot += 1
            if(arrayParola[2] == 'B'):
                try:
                    arAdv += embeddings_index[arrayParola[0].lower()]
                    arTOT += embeddings_index[arrayParola[0].lower()]
                    cadv += 1
                    ctot += 1
                except:
                    arAdv += arraySmooth
                    arTOT += arraySmooth
                    cadv += 1
                    ctot += 1
            if(arrayParola[2] == 'S'):
                try:
                    arNomi += embeddings_index[arrayParola[0].lower()]
                    arTOT += embeddings_index[arrayParola[0].lower()]
                    cnomi += 1
                    ctot += 1
                except:
                    arNomi += arraySmooth
                    arTOT += arraySmooth
                    cnomi += 1
                    ctot += 1
            if(arrayParola[2] == 'V'):
                try:
                    arVerb += embeddings_index[arrayParola[0].lower()]
                    arTOT += embeddings_index[arrayParola[0].lower()]
                    cverb += 1
                    ctot += 1
                except:
                    arVerb += arraySmooth
                    arTOT += arraySmooth
                    cverb += 1
                    ctot += 1
            else:
                try:
                    arTOT += embeddings_index[arrayParola[0].lower()]
                    ctot += 1
                except:
                    arTOT += arraySmooth
                    ctot += 1
        if(cnomi == 0):
            cnomi = 1
        if(cverb == 0):
            cverb = 1
        if(cadj == 0):
            cadj = 1
        if(cadv == 0):
            cadv = 1
        if(ctot == 0):
            ctot = 1
        arrayFrase = np.concatenate((arNomi/cnomi, arVerb/cverb, arAdj/cadj, arAdv/cadv, arTOT/ctot), axis=None)            
        arrayTokenizzatoLemmi.append(arrayFrase)
    return arrayTokenizzatoLemmi


def generaCombDPL(dataset = generaArrayTokenizzatoSurface(listPOS = ['A','S','V'])):
    embeddings_index = importaWordSpace2()
    combDataset = []
    for tweet in dataset:
        comb = [0.0]
        for parola in tweet:
            if(parola in embeddings_index):
                comb = np.concatenate((comb, embeddings_index[parola]), axis=None)
        combDataset.append(comb)
    return combDataset

def appendBowDPL( nomerappresentazione, dataset = generaArrayTokenizzatoSurface(listPOS = ['A','S','V']), file_name = "dataset.klp"):
    f = open(file_name, 'r')
    embeddings_index = importaWordSpace2()
    file_lines = []
    for index, x in enumerate(f.readlines()):
        string = " |BV:"+ nomerappresentazione + "| "
        for word in dataset[index]:
            try:
                vecPol = embeddings_index[word]
            except:
                vecPol = [0, 0, 0]
            pal = hashlib.md5(''.join(word)).hexdigest()
            string = string + "_" + pal +"_pos:"+ str(vecPol[0]) + " " + pal +"_neg:"+ str(vecPol[1]) + " " + pal +"_net:"+ str(vecPol[2]) + " "
        string = string + "|EV| "
        file_lines.append(''.join([x.strip(), string, '\n']))

    with open(file_name, 'w') as f:
        f.writelines(file_lines)


def generaCombSentix(dataset = prova2):
    embeddings_index = importaWordSpace4()
    combDataset = []
    for tweet in dataset:
        comb = np.zeros(4)
        norm = 0.0
        for parola in tweet:
            if(parola in embeddings_index):
                try:
                    comb = comb + embeddings_index[parola]
                except:
                    print parola
                    print embeddings_index[parola]
                norm += 1
        if(norm == 0):
            norm = 1
        combDataset.append(comb/norm)
    return combDataset

                

In [66]:
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
import numpy as np

def generaEmoji(dataset = X_train):
    embeddings_index = importaWordSpace3()
    combDataset = []
    for tweet in dataset:
        tknzr = TweetTokenizer(strip_handles=True)
        tweettoknz = tknzr.tokenize(tweet)
        comb = np.zeros(300)
        for parola in tweettoknz:
            if(parola in embeddings_index):
                print parola
                comb = comb + embeddings_index[parola]
        combDataset.append(comb)
    return combDataset

def feturesPunteggiatura(dataset = X_train):
    combDataset = []
    for tweet in dataset:
        charset = [0.0,0.0,0.0,0.0,0.0,0.0]
        numUpperCase = 0.0
        numchar = 0.0
        for char in tweet:
            numchar += 1
            if(char.istitle()):
                numUpperCase += 1
            if(char == "!"):
                charset[0] += 1
            if(char == "?"):
                charset[1] += 1
            if(char == "."):
                charset[2] += 1    
            if(char == ","):
                charset[3] += 1
            if(char == ";"):
                charset[4] += 1
            if(char == ":"):
                charset[5] += 1
        charset.append(numUpperCase/numchar)
        combDataset.append(charset)
    return combDataset 

In [67]:
from scipy.spatial import distance

#Combinazione Lineare delle parole del Word Space moltiplicate per la distanza dal centroide
#Per ogni Elemento del dataset ritorna la tupla (Bow, Var, Mean)
def generaIronySpecificBow(dataset, arrayCentroide = generaCombWordSpace(dataset=prova)):
    embeddings_index = importaWordSpace()
    combDataset = []
    for index, tweet in enumerate(dataset):
        comb = np.zeros(250)
        arrayDistanze = []
        for parola in tweet:
            if(parola in embeddings_index):
                comb = comb + embeddings_index[parola] * distance.cosine(embeddings_index[parola], arrayCentroide[index])
                arrayDistanze.append(distance.euclidean(embeddings_index[parola], arrayCentroide[index]))
        x = asarray(arrayDistanze)
        aList = (comb, np.var(x), np.mean(x))
        combDataset.append(tuple(aList))
    return combDataset

In [68]:
from scipy.spatial import distance

def calcolaDistanzaSmooth(dataset=arrayCompleto, embeddings_index = importaWordSpace(), arrayCentroide = generaCombWordSpace(dataset=prova)):
    arrayDistanzeDataset = []
    for index, tweet in enumerate(dataset):
        arrayDistanzetweet = []
        for word in tweet:
            if(word in embeddings_index):
                distanza = 1 - distance.cosine(arrayCentroide[index], embeddings_index[word])
                arrayDistanzetweet.append(distanza)
        arrayDistanzeDataset.append(np.mean(arrayDistanzetweet))
    return np.mean(arrayDistanzeDataset)

#calcolaDistanzaSmooth()
#Output: 0.4273651809245167

In [69]:
from scipy.spatial import distance
                

def appendBowIronySpecific(dataset, nomerappresentazione, file_name = "dataset.klp", embeddings_index = importaWordSpace(), arrayCentroide = generaCombWordSpace(dataset=prova)):
    f = open(file_name, 'r')
    smooth = 0.4273651809245167
    file_lines = []
    combDataset = []
    missingWord = 0.0
    totalWord = 0.0
    for index, x in enumerate(f.readlines()):
        string = " |BV:"+ nomerappresentazione + "| "
        arrayDistanze = []
        flag = 0
        distanzaMax = 0
        distanzaMin = 4
        for word in dataset[index]:
            #calcola distanza tra word e centroide[index]
            if(word in embeddings_index):
                distanza = 1 - distance.cosine(arrayCentroide[index], embeddings_index[word])
                if(distanza > distanzaMax):
                    distanzaMax = distanza
                if(distanza < distanzaMin):
                    distanzaMin = distanza
                arrayDistanze.append(1 - distance.cosine(arrayCentroide[index], embeddings_index[word]))
                string = string + "_" + hashlib.md5(''.join(word)).hexdigest() + ":"+ str(distanza)+" "
                flag = 1
            else:
                distanza = smooth
                missingWord += 1
            totalWord += 1
                
        string = string + "|EV| "
        '''
        if(flag == 1):
            file_lines.append(''.join([x.strip(), string, '\n']))
        else:
            file_lines.append(''.join([x.strip(), "", '\n']))
        '''
        file_lines.append(''.join([x.strip(), string, '\n']))
        x = asarray(arrayDistanze)
        if(np.isnan(np.var(x)) or np.isnan(np.mean(x))):
            aList = [-2,-2,-2,-2]
        else:
            aList = [np.var(x), np.mean(x), distanzaMin, distanzaMax]
        combDataset.append(aList)

    with open(file_name, 'w') as f:
        f.writelines(file_lines)
    print "Parole mancanti: " + str(missingWord/totalWord)
    return combDataset

In [70]:
import hashlib


def appendBow(dataset, nomerappresentazione, file_name = "dataset.klp", tupla = True):
    f = open(file_name, 'r')
    file_lines = []
    for index, x in enumerate(f.readlines()):
        string = " |BV:"+ nomerappresentazione + "| "
        for word in dataset[index]:
            if(tupla):
                string = string + "_" + hashlib.md5(''.join(word)).hexdigest() + ":1.0 "
            else:
                string = string + "_" + word + ":1.0 "
        string = string + "|EV| "
        file_lines.append(''.join([x.strip(), string, '\n']))

    with open(file_name, 'w') as f:
        f.writelines(file_lines) 
        
        


def appendDenseRapr(dataset, nomerappresentazione, file_name = "dataset.klp"):
    f = open(file_name, 'r')
    file_lines = []
    for index, x in enumerate(f.readlines()):
        string = " |BDV:"+ nomerappresentazione + "| "
        for num in dataset[index]:
            string = string + str(num) + " "
        string = string + "|EDV| "
        file_lines.append(''.join([x.strip(), string, '\n']))
        
    with open(file_name, 'w') as f:
        f.writelines(file_lines)


def appendDenseRaprTuple(dataset, nomerappresentazione, idTupla, file_name = "dataset.klp"):
    f = open(file_name, 'r')
    file_lines = []
    for index, x in enumerate(f.readlines()):
        string = " |BDV:"+ nomerappresentazione + "| "
        if( idTupla < 1):
            for num in dataset[index][idTupla]:
                string = string + str(num) + " "
        else:
            string = string + str(dataset[index][idTupla]) + " "
        string = string + "|EDV| "
        file_lines.append(''.join([x.strip(), string, '\n']))
        
    with open(file_name, 'w') as f:
        f.writelines(file_lines)

        
def padding(combDPL):
    massima = 0
    nuovoarray = []
    for array in combDPL:
        if len(array) > massima:
            massima = len(array)
    for array in combDPL:
        arrayzeri = np.zeros(massima - len(array))
        array2 = np.concatenate((array, arrayzeri), axis=None)
        nuovoarray.append(array2)
    return nuovoarray

### Append delle rappresentazioni KLP

In [71]:
combWS = generaCombWordSpace()
appendDenseRapr(combWS, "WSSurface",NOME_FILE)



comDPL = generaCombDPL2()
appendDenseRapr(comDPL, "combDPL",NOME_FILE)

comSentix = generaCombSentix()
appendDenseRapr(comSentix, "combSentix",NOME_FILE)

combEmoji = generaEmoji()
appendDenseRapr(combEmoji, "combEmoji",NOME_FILE)

featuresPuntegg = feturesPunteggiatura()
appendDenseRapr(featuresPuntegg, "featPunt",NOME_FILE)

In [72]:
meanVarArray = appendBowIronySpecific(prova, "bowIronySpecific", NOME_FILE)
appendDenseRapr(meanVarArray, "VarMean",NOME_FILE)

IronySpecificA = generaArrayTokenizzatoSurface(listPOS = ['A'])
meanVarArrayA = appendBowIronySpecific(IronySpecificA, "bowIronySpecificA", NOME_FILE)
appendDenseRapr(meanVarArrayA, "VarMeanA",NOME_FILE)

IronySpecificS = generaArrayTokenizzatoSurface(listPOS = ['S'])
meanVarArrayS = appendBowIronySpecific(IronySpecificS, "bowIronySpecificS", NOME_FILE)
appendDenseRapr(meanVarArrayS, "VarMeanS",NOME_FILE)

IronySpecificV = generaArrayTokenizzatoSurface(listPOS = ['V'])
meanVarArrayV = appendBowIronySpecific(IronySpecificV, "bowIronySpecificV", NOME_FILE)
appendDenseRapr(meanVarArrayV, "VarMeanV",NOME_FILE)

In [73]:

irony_specific_bow = generaIronySpecificBow(arrayCompleto)
appendDenseRaprTuple(irony_specific_bow, "IronySpecificWS", 0 ,NOME_FILE)
#appendDenseRaprTuple(irony_specific_bow, "Var", 1 ,NOME_FILE)
#appendDenseRaprTuple(irony_specific_bow, "Mean", 2 ,NOME_FILE)


In [74]:
appendBow(prova, "bowLemmi", NOME_FILE, tupla = False)
appendBowDPL(nomerappresentazione = "bowDPL", file_name = NOME_FILE)
appendBow(arrayBigrammiLemmi, "bowBigramLemmi", NOME_FILE)
appendBow(arrayBigrammiSurface, "bowBigramSurface", NOME_FILE)



appendBow(arrayBigrammiChar, "bow2gramChar", NOME_FILE)
appendBow(array3grammiChar, "bow3gramChar", NOME_FILE)
appendBow(array4grammiChar, "bow4gramChar", NOME_FILE)
appendBow(array5grammiChar, "bow5gramChar", NOME_FILE)

# Irony Corpus

In [76]:
from tqdm import tqdm

#Tipo 0 Surface
#Tipo 1 Lemma
def arrayIrony(path="../irony_corpus_sentences_processed.txt", ok=False, tipo=0, lower = True, listPOS = ['A','S','V']):
    arrayLemmi = open(path, "r").readlines()
    arrayTokenizzatoLemmi = []
    for text in arrayLemmi:
        arrayParoleLemm = text.split()
        arrayFrase = []
        for parola in arrayParoleLemm:
            arrayParola = parola.split("::")
            if((arrayParola[2] in listPOS) or ok):
                if(lower):
                    arrayFrase.append(arrayParola[tipo].lower())
                else:
                    arrayFrase.append(arrayParola[tipo])
        arrayTokenizzatoLemmi.append(arrayFrase)
    return arrayTokenizzatoLemmi

def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def generaDictNgram(arr, numNgram = 0):
    dic = dict()
    for line in tqdm(arr):
        if numNgram != 0:
            dic = merge_two_dicts(FreqDist(ngrams(line, numNgram)), dic)
        else:
            dic = merge_two_dicts(FreqDist(line), dic)
    return dic

## Genero dizionari con frequenza su file

##### NB: il codice qui sotto è da eseguire solo una volta (la prima, cambiando la tipologia di cella da markdown a code) per la generazione su file del dizionario delle frequenze delle parole nell' Irony Corpus

from nltk import ngrams, FreqDist
import pickle

arr = arrayIrony(ok=True)
arr2 = arrayIrony(tipo=1)

all_counts = dict()

for size in 0, 2, 3:
    if(size == 0):
        all_counts[size] = generaDictNgram(arr2, size)
    else:
        all_counts[size] = generaDictNgram(arr, size)
        
with open('unigrammi.pickle', 'wb') as handle:
    pickle.dump(all_counts[0], handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('bigrammi.pickle', 'wb') as handle:
    pickle.dump(all_counts[2], handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('trigrammi.pickle', 'wb') as handle:
    pickle.dump(all_counts[3], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [77]:
import pickle

with open('unigrammi.pickle', 'rb') as handle:
    unigramDict = pickle.load(handle)

with open('bigrammi.pickle', 'rb') as handle:
    bigramDict = pickle.load(handle)

with open('trigrammi.pickle', 'rb') as handle:
    trigramDict = pickle.load(handle)

In [78]:
import numpy as np

def appendBowIronyCorpus(dataset, nomerappresentazione, dizionarioFrequenze, file_name = "dataset.klp"):
    f = open(file_name, 'r')
    file_lines = []
    for index, x in enumerate(f.readlines()):
        string = " |BV:"+ nomerappresentazione + "| "
        for word in dataset[index]:
            if(word in dizionarioFrequenze):
                if(dizionarioFrequenze[word] >= 1):
                    string = string + "_" + hashlib.md5(''.join(word)).hexdigest() + ":" + str(np.log(dizionarioFrequenze[word] + 1))  + " "
        string = string + "|EV| "
        file_lines.append(''.join([x.strip(), string, '\n']))
    with open(file_name, 'w') as f:
        f.writelines(file_lines)
        
def appendBowBinaryIronyCorpus(dataset, nomerappresentazione, dizionarioFrequenze, file_name = "dataset.klp"):
    f = open(file_name, 'r')
    file_lines = []
    for index, x in enumerate(f.readlines()):
        string = " |BV:"+ nomerappresentazione + "| "
        for word in dataset[index]:
            if(word in dizionarioFrequenze):
                if(dizionarioFrequenze[word] >= 1):
                    string = string + "_" + hashlib.md5(''.join(word)).hexdigest() + ":1.0 "
        string = string + "|EV| "
        file_lines.append(''.join([x.strip(), string, '\n']))
    with open(file_name, 'w') as f:
        f.writelines(file_lines)
        

In [79]:
arrayTrigrammiSurface = generaArrayNgrams(arrayCompleto, 3)

appendBowIronyCorpus(prova, "bowIC", unigramDict, NOME_FILE)
appendBowIronyCorpus(arrayBigrammiSurface, "bowIC2gramSurface",bigramDict, NOME_FILE)
appendBowIronyCorpus(arrayTrigrammiSurface, "bowIC3gramSurface",trigramDict, NOME_FILE)

appendBowBinaryIronyCorpus(prova, "bowICBIN", unigramDict, NOME_FILE)
appendBowBinaryIronyCorpus(arrayBigrammiSurface, "bowICBIN2gramSurface",bigramDict, NOME_FILE)
appendBowBinaryIronyCorpus(arrayTrigrammiSurface, "bowICBIN3gramSurface",trigramDict, NOME_FILE)


In [80]:
def generaArrayDenso(dataset, dizionarioFrequenze):
    combDataset = []
    for index, tweet in enumerate(dataset):
        comb = np.zeros(4)
        intnumparl1 = 0.0
        intnumparl0 = 0.0
        massimo = 0.0
        minimo = 0.0
        for word in tweet:
            if(word in dizionarioFrequenze):
                intnumparl1 += 1
                comb[0] += dizionarioFrequenze[word]
                if(dizionarioFrequenze[word] > massimo):
                    massimo = dizionarioFrequenze[word]
                if(dizionarioFrequenze[word] < minimo or intnumparl0 == 0 ):
                    minimo = dizionarioFrequenze[word]
            intnumparl0 += 1
        if(intnumparl0 == 0):
            intnumparl0 = 1
        comb[0] = comb[0] / intnumparl0
        comb[1] = intnumparl1 / intnumparl0
        comb[2] = massimo
        comb[3] = minimo
        combDataset.append(comb)
    return combDataset

In [81]:
densoUnigram = generaArrayDenso(prova, unigramDict)
densoBigram = generaArrayDenso(arrayBigrammiSurface, bigramDict)
densoTrigram = generaArrayDenso(arrayTrigrammiSurface, trigramDict)


appendDenseRapr(densoUnigram, "densUnigram",NOME_FILE)
appendDenseRapr(densoBigram, "densBigram",NOME_FILE)
appendDenseRapr(densoTrigram, "densTrigram",NOME_FILE)

# Controllo del file prima dell'invio 

In [43]:
import pandas as pd
def controllaFileOutput(nomeoutput = "sanitycheck.tsv", pathTest = "../test_ironita2018_revnlt_processed.tsv", pathOutput = "../testOutput.tsv"):
    f = open(nomeoutput,"w+")
    test = pd.read_csv(pathTest, sep='\t')
    out = pd.read_csv(pathOutput, sep='\t')
    for x, line in test.iterrows():
        lineaOut = out.iloc[x]
        if(line["id"] == lineaOut["id"]):
            f.write(str(line["id"])+"\t"+str(lineaOut["irony"])+"\t"+str(lineaOut["sarcasm"])+"\t"+line["text"]+"\n")
        else:
            raise ValueError('Gli id '+ str(line["id"]) + " e " + str(lineaOut["id"]) + " non sono uguali \n Linea: " + str(x))   

In [44]:
controllaFileOutput()

### Softmax normalizzazione Kernel

In [111]:
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

scores = [0.6985084, 0.6826334]
softmax(scores)

array([0.50396867, 0.49603133])

In [45]:
! open .